In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from typing import TypedDict
from dotenv import load_dotenv

In [2]:
load_dotenv() 
model = ChatGroq(model="openai/gpt-oss-safeguard-20b", temperature=0, max_tokens=1024)

In [3]:
class BlogState(TypedDict):

    title: str
    outline: str
    content: str

In [4]:
def create_outline(state: BlogState) -> BlogState:

    title = state['title']

    prompt = f"Write a comprehensive outline for a blog post about: '{title}'"
    outline = model.invoke(prompt).content

    state['outline'] = outline
    return state

In [5]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f"Write a detailed blog post with the title '{title}' based on the following outline:\n{outline}"
    content = model.invoke(prompt).content
    state['content'] = content
    
    return state

In [ ]:
graph = StateGraph(BlogState)

graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()

In [11]:
intial_state = {'title': 'Strength of daffodil international university'}

final_state = workflow.invoke(intial_state)


In [12]:
print(final_state['outline'])

**Blog Post Outline – “The Strengths of Daffodil International University”**

| Section | Sub‑topics | Key Points / Examples |
|---------|------------|-----------------------|
| **1. Introduction** | • Hook: Why talk about DIU? <br>• Quick snapshot: location, founding year, mission | • “From a modest start in 2002 to a top‑tier private university in Bangladesh” <br>• Mission: “To produce globally competitive, ethically grounded professionals.” |
| **2. Academic Excellence** | • Diverse program portfolio <br>• Accreditation & quality assurance <br>• Faculty credentials | • 20+ undergraduate & graduate programs across Engineering, Business, IT, Social Sciences, Law, Medicine, etc. <br>• ISO 9001:2015, BQA, and international accreditations (e.g., ABET‑like for engineering). <br>• Faculty mix: PhDs from UK, USA, Australia; industry‑experienced adjuncts. |
| **3. Research & Innovation** | • Research centers & labs <br>• Funding & grants <br>• Publication record | • Centers: Center for Artif

In [13]:
print(final_state['content'])

# The Strengths of Daffodil International University

Daffodil International University (DIU) has become a beacon of higher education in Bangladesh. From its humble beginnings in 2002 to its current status as a top‑tier private university, DIU has consistently pushed the boundaries of academic excellence, research innovation, and community impact. In this post, we’ll walk through the key pillars that make DIU a powerhouse and why it’s worth a closer look—whether you’re a prospective student, a researcher, or a corporate partner.

---

## 1. Introduction

### Hook: Why Talk About DIU?

When you think of universities in South Asia, the first names that often come to mind are the public giants—Dhaka University, Jahangirnagar, and the like. Yet, a private institution founded just two decades ago has managed to carve out a reputation that rivals, and in some areas surpasses, these historic institutions. That’s the story of Daffodil International University.

### Quick Snapshot

- **Location